## Import packages

In [6]:
################### DO NOT ALTER ##################################################
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None # avoids harmless warnings
import json
import matplotlib.pyplot as plt
import statsmodels.api as sm

from datetime import datetime

from backend_v2 import * # import all functions from backend

################### DO NOT ALTER ##################################################

In [5]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


## Import modeling database

In [8]:
# read the database
# STUDENT INPUT REQUIRED - Modify path directly below for location of the final_database.csv file on your laptop/desktop
db = pd.read_csv(r'C:\Users\yshah\Documents\Babson\Ekimetrics Project\Modeling/final_database.csv')

In [9]:
################### DO NOT ALTER ##################################################
# rename date column and turn to date format
db = db.rename(columns={'week_starting_date':'date'})
db['date'] = pd.to_datetime(db['date'])

## specify the name of the target and date columns of the database
target_column = 'sales'
date_column = 'date'

# view available variables
db.columns
################### DO NOT ALTER ##################################################

Index(['date', 'sales', 'bing_brand_search_clicks', 'bing_brand_search_spend',
       'blackfriday_dummy', 'covid_trend', 'google_brand_search_clicks',
       'google_brand_search_imp', 'google_brand_search_spend',
       'google_display_clicks', 'google_display_imp', 'google_display_spend',
       'google_nonbrand_search_clicks', 'google_nonbrand_search_imp',
       'google_nonbrand_search_spend', 'google_video_clicks',
       'google_video_imp', 'google_video_spend', 'gwp_units_distributed',
       'launch_fragrance_trend', 'launch_makeup_trend',
       'launch_skincare_trend', 'market_sales', 'mothersday_dummy',
       'nongoogle_display_spend', 'nongoogle_video_spend',
       'pinterest_awareness_imp', 'pinterest_awareness_spend',
       'pinterest_conversion_imp', 'pinterest_conversion_spend',
       'print_coop_spend', 'print_pure_spend', 'samples_quantity_distributed',
       'snap_awareness_imp', 'snap_awareness_spend', 'snap_conversion_imp',
       'snap_conversion_spend', 'ti

## Create custom variables

In [10]:
# at this step create: dummies, offsets, combined vars, etc.
################### DO NOT ALTER #######################################################################

# windows
db = window_feat(db, start_end_week_pairs = [['2020-11-02', '2020-12-21']], feat_name = 'covid_holiday_period',)
db = window_feat(db, start_end_week_pairs = [['2020-12-14', '2020-12-21']], feat_name = 'covid_xmas',)

# timesplit
db = timesplit_feat(db, feature='tv_coop_spend', split_week='2020-04-13', after=True)
db = timesplit_feat(db, feature='print_pure_spend', split_week='2020-04-13', after=True)
db = timesplit_feat(db, feature='print_pure_spend', split_week='2020-04-13', after=False)

# aggregated variables
db['total_product_launch'] = db['launch_fragrance_trend'] + db['launch_makeup_trend'] + db['launch_skincare_trend']
    
# dummy variables
db['week_before_laborday'] = (np.where((db['date']=='2020-08-24')|
                                (db['date']=='2019-08-26'),
                                1,
                                0
                                )
                        )

db['week_before_vday'] = (np.where((db['date']=='2020-02-03')|
                                (db['date']=='2019-02-04'),
                                1,
                                0
                                )
                        )

db['july_promo'] = (np.where((db['date']=='2020-07-06')|
                                (db['date']=='2019-07-08'),
                                1,
                                0
                                )
                        )

db['mothersday_2019'] = (np.where((db['date']=='2019-05-06'),
                                1,
                                0
                                )
                        )

db['blackfriday_2019'] = (np.where((db['date']=='2019-11-25'),
                                1,
                                0
                                )
                        )

db['friends_and_family_august_promo'] = (np.where((db['date']=='2019-08-05'),
                                1.5,
                                0
                                )
                        )

db['friends_and_family_august_promo'] = (np.where((db['date']=='2020-08-03'),
                                1,
                                db['friends_and_family_august_promo']
                                )
                        )

db['flash_sale_promo'] = (np.where((db['date']=='2019-03-18')|
                                   (db['date']=='2019-04-15')|
                                   (db['date']=='2020-06-08'),
                                1,
                                0
                                )
                        )
db.columns
#########################  DO NOT ALTER ABOVE #########################################################################################

Index(['date', 'sales', 'bing_brand_search_clicks', 'bing_brand_search_spend',
       'blackfriday_dummy', 'covid_trend', 'google_brand_search_clicks',
       'google_brand_search_imp', 'google_brand_search_spend',
       'google_display_clicks', 'google_display_imp', 'google_display_spend',
       'google_nonbrand_search_clicks', 'google_nonbrand_search_imp',
       'google_nonbrand_search_spend', 'google_video_clicks',
       'google_video_imp', 'google_video_spend', 'gwp_units_distributed',
       'launch_fragrance_trend', 'launch_makeup_trend',
       'launch_skincare_trend', 'market_sales', 'mothersday_dummy',
       'nongoogle_display_spend', 'nongoogle_video_spend',
       'pinterest_awareness_imp', 'pinterest_awareness_spend',
       'pinterest_conversion_imp', 'pinterest_conversion_spend',
       'print_coop_spend', 'print_pure_spend', 'samples_quantity_distributed',
       'snap_awareness_imp', 'snap_awareness_spend', 'snap_conversion_imp',
       'snap_conversion_spend', 'ti

In [11]:
# THIRD, make the custom seasonality variable here (use examples above for reference):
# STUDENT INPUT REQUIRED - Wait to create this seasonality variable until after analyzing your initial model results and...
#                          ...you have completed the optimization of the fbig_imp and outdoor_impressions adstock and lag settings

# Create a combined seasonality variable (1 if any seasonal event is active, else 0)
db['seasonality_total'] = np.where(
    (db['xmas_dummy'] == 1) |
    (db['vday_dummy'] == 1) |
    (db['mothersday_dummy'] == 1) |
    (db['blackfriday_dummy'] == 1),
    1,
    0
)



## Build a Model and Evaluate

In [12]:
features_set = {
### Holidays ###
'blackfriday_2019': {},
'mothersday_2019': {'coeff': 1.2e+07},
'week_before_vday': {'adstock': 0.3, 'coeff': 9e+06},  # custom var
'july_promo': {},
'xmas_dummy': {},

### Market ###
'market_sales': {'coeff': 2.2e-02},

### COVID ###
'covid_holiday_period': {'coeff': -1.149319e+06},
'covid_xmas': {'coeff': -1.732296e+06},

### Product Launches ###
'total_product_launch': {'coeff': 2840.642889},
# 'launch_fragrance_trend': {},
# 'launch_makeup_trend': {},
# 'launch_skincare_trend': {},

### Promotions ###
'gwp_units_distributed': {'coeff': 9.734768},
'friends_and_family_august_promo': {'coeff': 4e+06},
'flash_sale_promo': {},

### Traditional Media ###
'tv_pure_spend': {'lag': 1, 'coeff': 1.1},
'tv_coop_spend': {'coeff': 4.5},
'tv_coop_spend_after_2020-04-13': {'coeff': -2.5},
'print_pure_spend_before_2020-04-13': {'coeff': 1.3},
'print_pure_spend_after_2020-04-13': {'lag': 1, 'coeff': 2},
'print_coop_spend': {'coeff': 8.655333},

### Digital Media ###
# Search
'google_brand_search_imp': {'coeff': 3.332862e-01},
'google_nonbrand_search_imp': {'coeff': 0.665232},
'bing_brand_search_clicks': {'coeff': 22},

# Social
'pinterest_awareness_imp': {'coeff': 1.692867e-01},
'pinterest_conversion_imp': {'coeff': 3.321005e-02},
'snap_awareness_imp': {'coeff': 0.04},
'snap_conversion_imp': {'coeff': 0.06},

# Video
'google_video_imp': {'coeff': 0.0308512},
'youtube_video_imp': {'coeff': 0.04582},
'nongoogle_video_spend': {'coeff': 0.01132},

# Display
'google_display_imp': {'coeff': 0.020604},
'nongoogle_display_spend': {'coeff': 0.014703},

# Trend & Sampling
'covid_trend': {},
'samples_quantity_distributed': {},

### OPTIMIZATION BLOCK ###
# FIRST optimize fbig_imp (try different values, like adstock: 0.3-0.6, lag: 0-2)
'fbig_imp': {
    'adstock': 0.4,
    'lag': 1
},

# SECOND optimize outdoor_impressions only after fbig_imp is finalized
'outdoor_impressions': {
    'adstock': 0.3,
    'lag': 1
},

# FOURTH, include your custom seasonality variable
'seasonality_total': {}
}


In [16]:
# Run model
ols_model, ols_lags, ols_adstocks, ols_coeffs_fixed, ols_model_coeffs, best_database = run_best_model(db, features_set, target_column)

# Create model output DataFrame
model_df = model_output(ols_model_coeffs, ols_adstocks, ols_lags, ols_model, features_set)

# Display
model_df


C:\Users\yshah\Documents\Babson\Ekimetrics Project\Modeling\backend_v2.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.    0.    0.    0.    1.    0.3   0.09  0.027 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.3   0.09  0.027
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.   ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_X.loc[:,feature] = adstock_transfo(model_X[feature], adstock)


,lag,adstock,saturation,coeff,t stat
blackfriday_2019,0,0.0,,3.765175e+06,3.194470
july_promo,0,0.0,,2.200969e+06,2.918106
xmas_dummy,0,0.0,,-1.190479e+06,-1.336691
flash_sale_promo,0,0.0,,2.538640e+06,4.145311
covid_trend,0,0.0,,5.895423e+04,7.325705
samples_quantity_distributed,0,0.0,,1.494406e+00,4.673811
fbig_imp,1,0.4,,6.743620e-02,4.554560
outdoor_impressions,1,0.3,,5.298869e-02,0.861899
seasonality_total,0,0.0,,-8.909664e+04,-0.184817
mothersday_2019,0,0.0,,1.200000e+07,0.000000


## To Paste in the Model sheet of the Analysis Tool

In [ ]:
# paste the output to the Model sheet of the Analysis Tool
#STUDENT ACTION REQUIRED: Paste the results that are in your clipboard to the Model tab (position cursor in cell G5) in the provided Analysis Workbook spreadsheet
model_df.to_clipboard(header = False) # this line puts the output in the clipboard, you can then paste it where needed

## To Paste in the InputSheet sheet of the Analysis Tool

In [ ]:
# Paste the output in the InputSheet sheet of the Analysis Tool.
#STUDENT ACTION REQUIRED: Paste the results that are in your clipboard to the InputSheet tab (position cursor in cell D3) in the provided Analysis Workbook spreadsheet
input_df = inputsheet(best_database, model_df)
input_df.to_clipboard(header = True, index = False) # this line puts the output in the clipboard, you can then paste it where needed